##### Efficient Aruco Detect

In [1]:
import cv2 as cv
import numpy as np
import math
from time import sleep


def detect_aruco(frame):    
    frame = cv.GaussianBlur(frame, (3, 3), 1)
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    dictionary = cv.aruco.getPredefinedDictionary(cv.aruco.DICT_4X4_1000)
    parameters = cv.aruco.DetectorParameters()
    corners, ids, _ = cv.aruco.detectMarkers(gray, dictionary, parameters=parameters)
    centers = []
    if len(corners):
        for corner in corners:
            center = np.mean(corner, axis=1, dtype=int)
            centers.append(center)
    ids = (np.ravel(ids)).tolist()
    return ids, centers


##### Arena Orientation Detect

In [2]:
ANGLE = -1

def correct_Orientation(frame):
    global ANGLE
    if ANGLE ==-1:
        dictionary = cv.aruco.getPredefinedDictionary(cv.aruco.DICT_4X4_1000)

        def get_ANGLE(marker_corners):
            p1 = marker_corners[0][0]
            p2 = marker_corners[0][1]
            ANGLE = np.degrees(np.arctan2(p2[1] - p1[1], p2[0] - p1[0]))
            return ANGLE

        marker_corners, marker_ids, _ = cv.aruco.detectMarkers(frame, dictionary)

        if marker_ids is not None:
            for i, marker_id in enumerate(marker_ids):
                ANGLE = get_ANGLE(marker_corners[i])
                break
    if ANGLE < 95 and ANGLE > 85:
        frame = cv.rotate(frame, cv.ROTATE_90_COUNTERCLOCKWISE)
    elif ANGLE < -85 and ANGLE > -95:
        frame = cv.rotate(frame, cv.ROTATE_90_CLOCKWISE)
    elif ANGLE < 185 and ANGLE > 175:
        frame = cv.rotate(frame, cv.ROTATE_90_CLOCKWISE)
        frame = cv.rotate(frame, cv.ROTATE_90_CLOCKWISE)
    
    return frame


##### Camera Matrix and Distortion Coefficients

In [3]:
def undistort_image(img, npz_file):
    data = np.load(npz_file)
    mtx = data["camMatrix"]
    dist = data["distCoef"]
    h, w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
    dst = cv.undistort(img, mtx, dist, None, newcameramtx)
    x, y, w, h = roi
    dst = dst[y : y + h, x : x + w]

    return dst


##### Open the Camera

In [4]:
def open_camera(cam_num):
    if cam_num == 0:
        try:
            cap = cv.VideoCapture("/dev/video0")
            if not cap.isOpened():
                raise Exception("Could not open /dev/video0")
        except:
            cap = cv.VideoCapture("/dev/video1")
            if not cap.isOpened():
                print(
                    "Could not open /dev/video1 either. Please check your device."
                )
    elif cam_num == 1:
        try:
            cap = cv.VideoCapture("/dev/video2")
            if not cap.isOpened():
                raise Exception("Could not open /dev/video2")
        except:
            cap = cv.VideoCapture("/dev/video3")
            if not cap.isOpened():
                print("Could not open /dev/video3 either. Please check your device.")
            

    desired_width = 1920
    desired_height = 1080
    cap.set(cv.CAP_PROP_FRAME_WIDTH, desired_width)
    cap.set(cv.CAP_PROP_FRAME_HEIGHT, desired_height)
    return cap


In [5]:
def arrange_clockwise(pts):
    pts = np.array(pts).reshape((4, 2))
    centroid = np.mean(pts, axis=0)
    ANGLEs = np.arctan2(pts[:, 1] - centroid[1], pts[:, 0] - centroid[0])
    pts = pts[np.argsort(ANGLEs)]
    pts = pts.tolist()
    for i in range(len(pts)):
        pts[i] = [pts[i]]
    return pts


##### Put Bounding Boxes and Events Name

In [6]:
def put_box_and_name(arena, event_boxes, events):
    # for i in range(0, len(event_boxes)):
    arena = cv.rectangle(
        arena,
        event_boxes[0],
        event_boxes[1],
        (0, 255, 0),
        3,
    )
    arena = cv.putText(
        arena,
        events,
        (event_boxes[0][0] + 100, event_boxes[0][1] + 50),
        cv.FONT_HERSHEY_COMPLEX,
        0.9,
        (0, 255, 0),
        3,
    )

    return arena


#####   Load the Model

In [7]:
from keras.models import load_model
from keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input

class_labels = [
    "combat",
    "destroyed_buildings",
    "empty",
    "fire",
    "human_aid_rehabilitation",
    "military_vehicles",
]
IMG_SIZE = (160, 160)
IMAGE_ORDER = ["A", "B", "C", "D", "E"]
predicted_class_list = []


# def predict_class(img_path):
#     loaded_model = load_model("/mnt/Storage/Dataset/model_HC.h5")
#     # print(loaded_model.summary())
#     img = image.load_img(img_path, target_size=IMG_SIZE)
#     img_array = image.img_to_array(img)
#     img_array = np.expand_dims(img_array, axis=0)
#     img_array = preprocess_input(img_array)
#     probabilities = loaded_model.predict(img_array)
#     predicted_class_index = np.argmax(probabilities)
#     pred = class_labels[predicted_class_index]
#     return pred


def predict_class(img):
    loaded_model = load_model("/mnt/Storage/Dataset/Yolo-cls/model_HC2.h5")
    # img = cv.resize(img, IMG_SIZE)
    # img_array = np.array(img)
    # img_array = image.img_to_array(img)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img_array = np.expand_dims(img, axis=0)
    img_array = preprocess_input(img_array)
    probabilities = loaded_model.predict(img_array)
    predicted_class_index = np.argmax(probabilities)
    pred = class_labels[predicted_class_index]

    return pred


2024-02-25 11:51:11.301162: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-25 11:51:11.487308: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 11:51:11.487352: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 11:51:11.514640: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-25 11:51:11.582963: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-25 11:51:11.584322: I tensorflow/core/platform/cpu_feature_guard.cc:1

##### Moves Generator

In [8]:
import numpy as np

map = np.array(
    [
        [0, None, None, None, None, None],
        [None, 1, 5, 6, 11, None],  #
        [None, 21, None, 24, None, 25],  #
        [None, 2, 4, 7, 10, None],  #
        [None, None, 22, 23, None, None],  #
        [None, None, 3, 8, 9, None],
    ]
)


def map_rotate(dir):
    global map
    if dir == 1:  # Clockwise
        map = np.flip(map.transpose(), 0)
    elif dir == -1:  # AntiClockwise
        map = np.flip(map.transpose(), 1)
    # print(map)


def mov_gen(path_str):
    global map
    orientation_vector = 0
    moves = ""
    path_counter = 0  # pointer for shortest_path string
    nodes_list = []  # list for contating all the nodes for each event visit
    nodes = []
    for i in range(0, len(path_str)):
        if path_str[i] == "P":
            node = int(path_str[path_counter + 1])
            if path_str[path_counter + 2] == "0" or path_str[path_counter + 2] == "1":
                node += int(path_str[path_counter + 2]) + 9
            nodes.append(node)
        elif path_str[i] == "A":
            nodes.append(21)
        elif path_str[i] == "B":
            nodes.append(22)
        elif path_str[i] == "C":
            nodes.append(23)
        elif path_str[i] == "D":
            nodes.append(24)
        elif path_str[i] == "E":
            nodes.append(25)
        elif path_str[i] == "s":
            nodes_list = nodes
            nodes = []

        path_counter += 1

    print(nodes_list)

    line_follow_time = 0
    forward_time = 0
    backward_time = 0
    right_time = 0
    left_time = 0

    for i in range(0, len(nodes_list) - 1):
        node1 = nodes_list[i]
        node2 = nodes_list[i + 1]
        node0 = 0
        if i > 0:
            node0 = nodes_list[i - 1]
        print(str(node1) + " --> " + str(node2) + " : ", end="")
        pos_num1 = np.argwhere(map == node1)
        pos_num2 = np.argwhere(map == node2)
        # print("\npos_num1", pos_num1)
        # print("pos_num2", pos_num2)
        if node1 in [21, 22, 23, 24, 25]:
            if node0 in [1, 4, 7, 6, 11] and [node0, node1] != [7, 24]:
                moves += str(chr(64 + (node1 - 20))) + "z"
            else:
                moves += str(chr(96 + (node1 - 20))) + "z"
        else:
            moves += "l" + str(line_follow_time)

        if (
            abs(pos_num1[0][0] - pos_num2[0][0]) <= 2
            and abs(pos_num1[0][1] - pos_num2[0][1]) <= 2
            and (pos_num1[0][0] == pos_num2[0][0] or pos_num1[0][1] == pos_num2[0][1])
            and [node1, node2] != [0, 1]
        ):
            if (
                pos_num1[0][0] + 1 == pos_num2[0][0]
                or pos_num1[0][0] + 2 == pos_num2[0][0]
            ):
                print("Right")
                moves += "r" + str(right_time)
                map_rotate(1)
            elif (
                pos_num1[0][0] == pos_num2[0][0] + 1
                or pos_num1[0][0] == pos_num2[0][0] + 2
            ):
                print("Left")
                moves += "a" + str(left_time)
                map_rotate(-1)
            elif (
                pos_num1[0][1] + 1 == pos_num2[0][1]
                or pos_num1[0][1] + 2 == pos_num2[0][1]
            ):
                print("forward")
                moves += "f" + str(forward_time)
            elif (
                pos_num1[0][1] == pos_num2[0][1] + 1
                or pos_num1[0][1] == pos_num2[0][1] + 2
            ):
                print("U-Turn")
                moves += "u" + str(backward_time)
                map_rotate(1)
                map_rotate(1)
            else:
                print("none1")

        elif [node0, node1, node2] == [6, 11, 25]:
            print("Forward")
            moves += "f" + str(forward_time)
            map_rotate(1)
        elif [node0, node1, node2] == [10, 11, 25]:
            print("Right")
            moves += "c" + str(forward_time)
            map_rotate(1)
            map_rotate(1)
        elif [node1, node2] == [25, 11]:
            print("U-turn")
            moves += "u" + str(forward_time)
            map_rotate(1)
        elif [node0, node1, node2] == [2, 1, 0] or [node0, node1, node2] == [21, 1, 0]:
            print("Left")
            moves += "a" + str(forward_time)
            map_rotate(-1)
        elif [node0, node1, node2] == [5, 1, 0]:
            print("Forward")
            moves += "f" + str(forward_time)
        elif [node1, node2] == [0, 1]:
            print("Forward")
            moves = moves[:-2]
        else:
            print("none2")

    moves += "ss"
    return moves


##### Path Planning

In [9]:
import heapq


def path_plan(priority_string):
    graph = {
        "P0": {"P1": 27},
        "P1": {"P0": 27, "P2": 97, "P5": 50, "A": 35},
        "P2": {"P4": 52, "P1": 97, "A": 62},
        "P3": {"P4": 103, "P2": 160, "B": 45},
        "P4": {"P3": 103, "P2": 52, "B": 60, "P7": 51, "P5": 100},
        "P5": {"P6": 57, "P1": 50, "P4": 100},
        "P6": {"P7": 100, "P11": 40, "P5": 57, "D": 38},
        "P7": {"P6": 100, "P4": 51, "D": 62, "P8": 103, "P10": 38, "C": 65},
        "P8": {"P7": 103, "C": 41, "P3": 50, "P9": 38},
        "P9": {"P8": 38, "P10": 102},
        "P10": {"P9": 102, "P11": 101},
        "P11": {"P10": 101, "E": 71, "P6": 40},
        "A": {"P1": 35, "P2": 62},
        "B": {"P3": 45, "P4": 60},
        "C": {"P7": 65, "P8": 41},
        "D": {"P7": 62, "P6": 38},
        "E": {"P11": 71},
    }

    start_node = "P0"

    def dijkstras(start, target):
        # Initialize distances to all nodes as infinity
        distances = {node: float("inf") for node in graph}
        distances[start] = 0  # Distance from start node to itself is 0

        # Priority queue to hold nodes to visit
        priority_queue = [(0, start)]  # (distance, node)

        # Dictionary to keep track of the shortest path
        previous_nodes = {}

        while priority_queue:
            # Pop the node with the smallest distance
            current_distance, current_node = heapq.heappop(priority_queue)

            # If we've reached the target node, we can reconstruct the path and return it
            if current_node == target:
                path = []
                while current_node in previous_nodes:
                    path.insert(0, current_node)
                    current_node = previous_nodes[current_node]
                path.insert(0, start)
                return path

            # Check distances to neighbors of the current node
            for neighbor, weight in graph[current_node].items():
                distance = current_distance + weight

                # If a shorter path is found, update distance and previous node
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous_nodes[neighbor] = current_node
                    heapq.heappush(priority_queue, (distance, neighbor))

        # If no path is found
        return []

    node_priority = priority_string

    path = [start_node]
    for i in range(len(node_priority)):
        current_node = path[-1]
        next_node = node_priority[i]
        shortest_path = dijkstras(current_node, next_node)
        path.extend(shortest_path[1:])
    shortest_path_to_P0 = dijkstras(path[-1], start_node)
    path.extend(shortest_path_to_P0[1:])
    path_str = "".join(path) + "s"

    return path_str


##### Sort According to Priority

In [10]:
def sort_priority(events_list):
    priority_order = [
        "fire",
        "destroyed_buildings",
        "human_aid_rehabilitation",
        "military_vehicles",
        "combat",
        "empty",
    ]
    events_dict = {}
    priority_string = ""
    for i in range(0, len(events_list)):
        if events_list[i] != "empty":
            events_dict[chr(65 + i)] = events_list[i]
    for i in priority_order:
        for j in events_dict:
            if i == events_dict[j]:
                priority_string += list(events_dict.keys())[
                    list(events_dict.values()).index(i)
                ]
                break

    return events_dict, priority_string


# print(
#     sort_priority(
#         ["combat", "fire", "military_vehicles", "destroyed_buildings", "empty"]
#     )
# )


##### Extract the events

In [11]:
def event_extract(arena):
    # cv.imshow("arena",arena)
    extracted_events = []
    ids, centers = detect_aruco(arena)
    roi_images_array = []
    event_corners = []
    event_boxes = []
    events = []
    marked_arena = arena.copy()
    event_arcs = [21, 29, 30, 34, 48]
    for i in event_arcs:
        for j in range(0, len(ids)):
            if ids[j] == i:
                point1 = (centers[j][0][0] - 10, centers[j][0][1] - 40)
                point2 = (
                    centers[j][0][0] - 160,
                    centers[j][0][1] + 110,
                )

                top_left = (min(point1[0], point2[0]), min(point1[1], point2[1]))
                bottom_right = (max(point1[0], point2[0]), max(point1[1], point2[1]))
                roi = arena[
                    top_left[1] : bottom_right[1], top_left[0] : bottom_right[0]
                ]
                # print(
                #     "ROI", [top_left[1], bottom_right[1], top_left[0], bottom_right[0]]
                # )
                roi_images_array.append(roi)
                roi_gray = cv.cvtColor(roi.copy(), cv.COLOR_BGR2GRAY)
                kernel = np.ones((5, 5), np.uint8)
                roi_denoise = cv.fastNlMeansDenoising(
                    src=roi_gray,
                    dst=None,
                    h=10,
                    templateWindowSize=7,
                    searchWindowSize=21,
                )
                roi_dialate = cv.dilate(
                    roi_denoise, kernel, iterations=1
                )  # decrease iterations if image is too bright
                thresh = cv.adaptiveThreshold(
                    roi_dialate,
                    255,
                    cv.ADAPTIVE_THRESH_MEAN_C,
                    cv.THRESH_BINARY_INV,
                    11,
                    -2,
                )
                # if(i==29):
                #     cv.imshow("thresh" + str(j), thresh)
                #     cv.imshow("roi_denoise" + str(j), roi_denoise)

                contours, _ = cv.findContours(
                    thresh, cv.RETR_LIST, cv.CHAIN_APPROX_NONE
                )
                for cnt in contours:
                    approx = cv.approxPolyDP(cnt, 0.01 * cv.arcLength(cnt, True), True)
                    if len(approx) == 4:
                        x1, y1, w, h = cv.boundingRect(cnt)
                        x2 = x1 + top_left[0]
                        y2 = y1 + top_left[1]
                        ratio = float(w) / h
                        area = w * h
                        if (
                            ratio >= 0.8
                            and ratio <= 1.2
                            and area > 8000
                            and area < 11000
                        ):
                            event_boxes.append([[x2, y2], [x2 + w, y2 + h]])
                            event_corners = []
                            for k in range(0, 4):
                                event_corners.append(approx[k][0])
                            # print("eventcorners", event_corners)

                            pts = np.reshape(event_corners, (-1, 1, 2))
                            if len(pts) != 0:
                                pts = arrange_clockwise(pts)
                                width = 160
                                height = 160
                                dstPts = [
                                    [0, 0],
                                    [width, 0],
                                    [width, height],
                                    [0, height],
                                ]
                                matrix = cv.getPerspectiveTransform(
                                    np.float32(pts), np.float32(dstPts)
                                )
                                cropped_event = cv.warpPerspective(
                                    roi, matrix, (int(width), int(height))
                                )
                                extracted_events.append(cropped_event)
                                cv.imshow(str(i), cropped_event)
                                cv.imwrite("images/bonus_v5_" + str(i) + ".jpg", cropped_event)
                                # predected_event = predict_class(cropped_event)
                                # events.append(predected_event)
                                # marked_arena = put_box_and_name(
                                #     marked_arena,
                                #     [[x2, y2], [x2 + w, y2 + h]],
                                #     predected_event,
                                # )
                                # print(str(i) + " : ", predected_event)
                                if cv.waitKey(1) & 0xFF == ord("q"):
                                    break

        # marked_arena = cv.polylines(arena.copy(), pts, 1, (0, 255, 0), 3)
    # for i in range(0, len(extracted_events)):
    #     cv.imshow(str(event_arcs[i]), extracted_events[i])
    #     cv.imwrite("Event_1" + str(event_arcs[i]) + ".jpg", extracted_events[i])
    #     events.append(predict_class(extracted_events[i]))
    #     print(str(i) + " : ", events[i])
    return extracted_events, event_boxes, events, marked_arena


##### Recording and processing

In [12]:
cap = open_camera(1)

extracted_events = []

while len(extracted_events) < 5:
    _, frame = cap.read()
    # frame = cv.imread(
    #     "/mnt/Storage/Projects/E-YRC/EYRC_2023/Task_5/Task_5A/Undistorted.jpg"
    # )
    frame = undistort_image(
        frame,
        "/mnt/Storage/Projects/E-YRC/EYRC_2023/Task_5/Task_5A/MultiMatrix.npz",
    )
    frame = correct_Orientation(frame)[450:1500, 0:1080]
    cv.imshow("frame_in", frame)
    extracted_events, event_boxes, events, marked_arena = event_extract(frame)
    print(len(extracted_events))

# # event_arcs = {"A": 21, "B": 29, "C": 30, "D": 34, "E": 48}


# # print("event_boxes", event_boxes)
# # print("events", events)
# # marked_arena = put_box_and_name(
# #     frame,
# #     event_boxes,
# #     events,
# # )
# # events_dict, priority_string = sort_priority(events)
# # print("events_dict", events_dict)
# # print("priority_string", priority_string)
# # path_str = path_plan(priority_string)
# # moves_str = mov_gen(path_str)
# # print("path_str", path_str)
# # print("moves_str", moves_str)

# # marked_arena = cv.resize(marked_arena, (900, 1000))


# # while 1:
# #     cv.imshow("marked_arena", marked_arena)
# #     if cv.waitKey(1) & 0xFF == ord("q"):
# #         break
# cap.release()
# cv.destroyAllWindows()


#####  Send the Move Instructions to the bot

In [ ]:
# import socket

# esp32_ip = "192.168.1.6"  # Replace with the actual IP address of your ESP32
# client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# server_address = (esp32_ip, 80)
# client_socket.connect(server_address)
# message = moves_str
# while 1:
#     client_socket.sendall(message.encode())
#     data = client_socket.recv(1024).decode()
#     print("Received:", data)
#     if data == "Instructions Recieved":
#         break
    
# client_socket.close()


In [ ]:
# import rasterio
# import numpy as np
# import pandas as pd
# import csv


# def update_csv(bot_aruco_centre):
#     with rasterio.open("/mnt/Storage/Projects/E-YRC/EYRC_2023/Task_6/Arena_Image_modified.tif") as src:
#         # Get the bounds of the raster
#         left, bottom, right, top = src.bounds

#         # Get the resolution (in degrees) of a pixel
#         xres = (right - left) / src.width
#         yres = (top - bottom) / src.height

#         # Calculate the longitude and latitude of the center of the pixel
#         lon = left + xres / 2 + bot_aruco_centre[1]-10 * xres
#         lat = top - yres / 2 - bot_aruco_centre[0] * yres

#     lat_lon = [lat,lon]
#     live_loc_csv = (
#         "/mnt/Storage/Projects/E-YRC/EYRC_2023/Task_4/Task_4B/Live_location.csv"
#     )
#     df = pd.read_csv(
#         "/mnt/Storage/Projects/E-YRC/EYRC_2023/Task_4/Task_4B/lat_long (1).csv"
#     )
#     for __, row in df.iterrows():
#         lat_lon = [float(row["lat"]), float(row["lon"])]
#         with open(live_loc_csv, "w+", newline="") as csvfile:
#             csv_writer = csv.writer(csvfile)
#             csv_writer.writerow(["lat", "lon"])

#         with open(live_loc_csv, "a", newline="") as csvfile:
#             csv_writer = csv.writer(csvfile)
#             csv_writer.writerow(lat_lon)
#             csvfile.close()


In [ ]:
import csv
import math


def get_nearest_lat_lon(bot_aruco_centre):
    with open("/mnt/Storage/Downloads/PixelstoLatLon.csv", "r") as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip the header

        min_distance = float("inf")
        nearest_lat_lon = None

        for row in csv_reader:
            px, py, lat, lon = [float(val) for val in row]

            distance = math.sqrt(
                (px - bot_aruco_centre[0]) ** 2 + (py - bot_aruco_centre[1]) ** 2
            )

            if distance < min_distance:
                min_distance = distance
                nearest_lat_lon = (lat, lon)

    lat_lon = nearest_lat_lon
    live_loc_csv = (
        "/mnt/Storage/Projects/E-YRC/EYRC_2023/Task_4/Task_4B/Live_location.csv"
    )

    with open(live_loc_csv, "w+", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["lat", "lon"])

    with open(live_loc_csv, "a", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(lat_lon)
        csvfile.close()


##### QGIS Mapping

In [ ]:
cap = open_camera(1)
BOT_ARUCO_ID = 100


# def read_csv(min_dist_Aruco_id):
#     lat_lon = []
#     live_loc_csv = (
#         "/mnt/Storage/Projects/E-YRC/EYRC_2023/Task_4/Task_4B/Live_location.csv"
#     )
#     df = pd.read_csv(
#         "/mnt/Storage/Projects/E-YRC/EYRC_2023/Task_4/Task_4B/lat_long (1).csv"
#     )
#     for __, row in df.iterrows():
#         if row["id"] == min_dist_Aruco_id:
#             lat_lon = [float(row["lat"]), float(row["lon"])]
#             with open(live_loc_csv, "w+", newline="") as csvfile:
#                 csv_writer = csv.writer(csvfile)
#                 csv_writer.writerow(["lat", "lon"])

#             with open(live_loc_csv, "a", newline="") as csvfile:
#                 csv_writer = csv.writer(csvfile)
#                 csv_writer.writerow(lat_lon)
#                 csvfile.close()
#     return lat_lon


Const_ids_Check = []
Const_corner_Check = []
BOT_ARUCO_ID = 100
Const_centres = []
Const_ids = []

cnt = 0
while 1:
    _, frame = cap.read()
    frame = undistort_image(
        frame,
        "/mnt/Storage/Projects/E-YRC/EYRC_2023/Task_5/Task_5A/MultiMatrix.npz",
    )
    frame = correct_Orientation(frame)[450:1500, 0:1080]

    Const_ids, Const_centres = detect_aruco(frame)
    if len(Const_centres) > 49 and (BOT_ARUCO_ID in Const_ids):
        print("Const_centres", Const_centres)
        print("Const_ids", Const_ids)
        print("len(Const_ids)", len(Const_ids))
        break
    # else:
    #     cnt = 0
    #     Const_corner_Check = Const_centres.copy()
    #     Const_ids_Check = Const_ids.copy()
    #     print("Const_centres", Const_centres)
    #     print("Const_ids", Const_ids)
    #     print("len(Const_ids)", len(Const_ids))
    #     # print("len(Const_centres)", len(Const_centres))
    #     print("")


bot_aruco_centre = []
for i in range(0, len(Const_ids)):
    if Const_ids[i] == BOT_ARUCO_ID:
        bot_aruco_centre = np.ravel(Const_centres[i]).tolist()
        break
while 1:
    _, arena = cap.read()
    arena = undistort_image(
        arena,
        "/mnt/Storage/Projects/E-YRC/EYRC_2023/Task_5/Task_5A/MultiMatrix.npz",
    )
    arena = correct_Orientation(arena)[450:1500, 0:1080]
    ids, centres = detect_aruco(arena)

    for i in range(0, len(ids)):
        if ids[i] == BOT_ARUCO_ID:
            bot_aruco_centre = np.ravel(centres[i]).tolist()
            break

    print("bot_aruco_centre", bot_aruco_centre)
    get_nearest_lat_lon(bot_aruco_centre)
    # Const_centres[Const_ids.index(BOT_ARUCO_ID)] = centres[ids.index(BOT_ARUCO_ID)]
    # for i in ids:
    #     if i == BOT_ARUCO_ID:
    #         _, ArUco_corners2 = detect_ArUco_details(arena)
    #         if i in ArUco_corners2.keys():
    #             for j in ArUco_corners2[i]:
    #                 cntr_x += j[0]
    #                 cntr_y += j[1]
    #             bot_aruco_centre = [cntr_x // 4, cntr_y // 4]
    #             Aruco_centres.append(bot_aruco_centre)
    #     else:
    #         for j in ArUco_corners[i]:
    #             cntr_x += j[0]
    #             cntr_y += j[1]
    #         Aruco_centres.append([cntr_x // 4, cntr_y // 4])

    # print("Aruco_centres", Aruco_centres)
    # print("Aruco_ids", Aruco_ids)

    # for i in range(0, len(Const_centres)):
    #     if Const_ids[i] == BOT_ARUCO_ID:
    #         print("Const_centres[i][0]", Const_centres[i][0])
    #         arena = cv.circle(
    #             arena,
    #             Const_centres[i][0],
    #             10,
    #             (255, 0, 0),
    #             -1,
    #         )
    # else:
    #     arena = cv.circle(
    #         arena,
    #         (Const_centres[i][0][0], Const_centres[i][0][1]),
    #         5,
    #         (0, 255, 0),
    #         -1,
    #     )
    # distance_min = 10000
    # min_dist_Aruco = []
    # min_dist_Aruco_id = 0
    # for i in range(0, len(Const_ids)):
    #     distance = math.sqrt(
    #         (bot_aruco_centre[0] - Const_centres[i][0][0]) ** 2
    #         + (bot_aruco_centre[1] - Const_centres[i][0][1]) ** 2
    #     )
    #     if (
    #         distance_min > distance
    #         and distance > 2
    #         and distance < 80
    #         and Const_ids[i] != BOT_ARUCO_ID
    #     ):
    #         min_dist_Aruco = Const_centres[i][0]
    #         distance_min = distance
    #         min_dist_Aruco_id = Const_ids[i]

    # print("min_dist_Aruco", distance_min)
    # print("min_dist_Aruco_id", min_dist_Aruco_id)
    # print("Lat lon", read_csv(min_dist_Aruco_id))
    # lat_lon = read_csv(min_dist_Aruco_id)
    # arena = cv.circle(
    #     arena, (min_dist_Aruco[0], min_dist_Aruco[1]), 15, (0, 0, 255), -1
    # )
    arena = cv.resize(arena, (900, 900))
    cv.imshow("Arena", arena)
    if cv.waitKey(1) & 0xFF == ord("q"):
        break
cap.release()
cv.destroyAllWindows()


Could not open /dev/video3 either. Please check your device.


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
import cv2
import cv2.aruco as aruco
import numpy as np


def find_aruco_center(image_path, aruco_id):
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    marker_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_1000)
    parameters = cv2.aruco.DetectorParameters()
    corners, ids, _ = cv2.aruco.detectMarkers(gray, marker_dict, parameters=parameters)

    if ids is not None:
        for i in range(len(ids)):
            if ids[i] == aruco_id:
                c = corners[i][0]
                # Calculate the center coordinates
                center_x = int(np.mean(c[:, 0]))
                center_y = int(np.mean(c[:, 1]))
                return (center_x, center_y)

    return None


# Test the function
center = find_aruco_center("/mnt/Storage/Downloads/Arena_Image.jpg", 32)
if center:
    print(f"The center of ArUco marker 32 is at {center}.")
else:
    print("ArUco marker 32 was not found in the image.")


The center of ArUco marker 32 is at (574, 542).


In [1]:
import pandas as pd
import cv2

# Load the CSV file into a DataFrame
df = pd.read_csv("/mnt/Storage/Projects/E-YRC/EYRC_2023/Task_6/PixelstoLatLon.csv")

# Load an image
img = cv2.imread("/mnt/Storage/Downloads/Arena_Image.jpg")

# Plot red dots on the image according to x, y coordinates from the CSV file
for index, row in df.iterrows():
    cv2.circle(
        img, (int(row["x"]), int(row["y"])), radius=5, color=(0, 0, 255), thickness=-2
    )

# Display the image
img = cv2.resize(img, (900, 900))
cv2.imshow("Image with Dots", img)
cv2.imwrite("Image with Dots.jpg", img)

# Wait for a key press and then close the image window
cv2.waitKey(5000000)
cv2.destroyAllWindows()


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


In [ ]:
import math

# Given values
stroke = 400  # in mm
n = 4.5  # ratio of connecting rod length to crank radius
rpm = 200  # rotations per minute
theta_degrees = 30  # angle in degrees

# Calculated values
r = stroke / 2  # crank radius in mm
r_meters = r / 1000  # crank radius in meters
omega = 2 * math.pi * rpm / 60  # angular velocity in rad/s
theta_radians = math.radians(theta_degrees)  # angle in radians

# Velocity calculation
v = omega * r_meters * math.sqrt(1 - (math.cos(theta_radians) / n) ** 2)

print(f"The velocity of the slider is {v} m/s.")


In [4]:
import cv2
import numpy as np
import pyperclip


# Function to handle mouse events
def draw_circle(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDBLCLK:
        cv2.circle(
            img, (x, y), 5, (0, 0, 255), -1
        )  # Draw a red circle at the clicked position
        coordinates = f"({x}, {y})"
        print(f"Pixel coordinates: {coordinates}")  # Print the pixel coordinates
        pyperclip.copy(str(coordinates))  # Copy the pixel coordinates to the clipboard


# Load an image
img = cv2.imread("/mnt/Storage/Downloads/Arena_Image.jpg", cv2.IMREAD_COLOR)

# Create a named window that can be resized
cv2.namedWindow("image", cv2.WINDOW_NORMAL)

# Set the callback function for any mouse event
cv2.setMouseCallback("image", draw_circle)

while True:
    cv2.imshow("image", img)
    if cv2.waitKey(20) & 0xFF == 27:  # Break the loop when 'ESC' key is pressed
        break

cv2.destroyAllWindows()


Pixel coordinates: (885, 499)
Pixel coordinates: (506, 488)
Pixel coordinates: (708, 498)
Pixel coordinates: (804, 499)
Pixel coordinates: (615, 495)
Pixel coordinates: (559, 495)
Pixel coordinates: (663, 497)
Pixel coordinates: (759, 500)
Pixel coordinates: (846, 501)
Pixel coordinates: (119, 480)
Pixel coordinates: (329, 486)
Pixel coordinates: (238, 484)
Pixel coordinates: (427, 488)
Pixel coordinates: (178, 481)
Pixel coordinates: (282, 485)
Pixel coordinates: (383, 487)
Pixel coordinates: (468, 488)
Pixel coordinates: (504, 676)
Pixel coordinates: (504, 584)
Pixel coordinates: (505, 531)
Pixel coordinates: (505, 632)
Pixel coordinates: (883, 677)
Pixel coordinates: (713, 677)
Pixel coordinates: (806, 679)
Pixel coordinates: (611, 678)
Pixel coordinates: (557, 677)
Pixel coordinates: (664, 677)
Pixel coordinates: (763, 678)
Pixel coordinates: (849, 676)
Pixel coordinates: (121, 696)
Pixel coordinates: (328, 686)
Pixel coordinates: (417, 682)
Pixel coordinates: (233, 692)
Pixel coor